In [9]:
import os
import io
import re
import ast
import sys
import time
import json
import dotenv
import base64
import importlib
from pathlib import Path

from google import genai
from google.genai import types

import gcp_utils

In [2]:
dotenv.load_dotenv()

True

In [3]:
free_api_key = os.getenv("free_api_key")
billed_api_key = os.getenv("billed_api_key")

client = genai.Client(api_key=billed_api_key) 

### Gemini Input Prep.

In [4]:
# output jsonl file name, yani Gemini modeline verilecek input
OUTPUT_FILE = "gemini_batch_input_v0.jsonl"

In [5]:
prompt = f"""
Tell me 3 baby boy names that start with Er"""

print("Prompt:", prompt)

Prompt: 
Tell me 3 baby boy names that start with Er


In [6]:
## create jsonl file

n_repeat = 100


with open(OUTPUT_FILE, 'a', encoding='utf-8') as f:
    for idx in range(n_repeat):

        try:

            # Step.1 Construct prompt
            prompt = f"""Tell me 3 baby boy names that start with Er"""
            
            # Step.2 Create batch request
            batch_request = {
                    "key": f"{idx}",
                    "request": {
                        "contents": [
                            {
                                "parts": [
                                    {"text": prompt}
                                ]
                            }
                        ],
                    }
                }
            

            # Step.3 Write request to the jsonl file
            f.write(json.dumps(batch_request, ensure_ascii=False) + "\n")

        except:

            print(f"An error occured for")

### Gemini Start Job

In [8]:
# 1. Load the prepared jsonl file
uploaded_jsonl_file = client.files.upload(
    file=OUTPUT_FILE,
    config=types.UploadFileConfig(display_name='my_PII_first_request_upload', mime_type='jsonl')
)

# 2. Create batch job
file_batch_job = client.batches.create(
    model=os.getenv("GEMINI_MODEL"),
    src=uploaded_jsonl_file.name,
    config={
        'display_name': "my_PII_first_request_job",
    },
)

print(f"Created batch job: {file_batch_job.name}")

Created batch job: batches/ixfnf86mx2e554pe5uuo2ow8viopkgh7qwlx


### Get Status of Job

In [11]:
gcp_utils.list_all_batch_jobs(client)

Job Name: batches/ixfnf86mx2e554pe5uuo2ow8viopkgh7qwlx
Job Status: JOB_STATE_PENDING
Job Creation Time: 2026-01-19 20:39:48.227732+00:00
------------------------------
Job Name: batches/e04c45jry7riixqyx8fh3lyl1jb0qxng2ez7
Job Status: JOB_STATE_SUCCEEDED
Job Creation Time: 2026-01-02 01:58:25.600100+00:00
------------------------------
Job Name: batches/ea2evcbvm6e0o5aqq7gpgc4weujrtnstr08c
Job Status: JOB_STATE_SUCCEEDED
Job Creation Time: 2025-12-18 21:43:36.393371+00:00
------------------------------
Job Name: batches/brgbfm8p5pjncw99gnl41qw8ljnoziix5g0w
Job Status: JOB_STATE_SUCCEEDED
Job Creation Time: 2025-12-18 20:55:01.668785+00:00
------------------------------
Job Name: batches/xbkd0pyauy1y35i8lcmu98molekqmba7thrr
Job Status: JOB_STATE_SUCCEEDED
Job Creation Time: 2025-12-18 16:12:47.970228+00:00
------------------------------
Job Name: batches/6nrn91xkstu16lka7yo5isb4i0l9kznibd83
Job Status: JOB_STATE_SUCCEEDED
Job Creation Time: 2025-12-18 00:46:50.916629+00:00
-------------

In [12]:
def fetch_and_parse_batch_results(client, job_id):
    # 1. Retrieve the final status of the batch job
    batch_job = client.batches.get(name=job_id)
    
    if batch_job.state.name != 'JOB_STATE_SUCCEEDED':
        print(f"Batch job is not ready. Current state: {batch_job.state.name}")
        return
    
    # If batch job was created with a file
    elif batch_job.dest and batch_job.dest.file_name:
        # Results are in a file
        result_file_name = batch_job.dest.file_name
        print(f"Results are in file: {result_file_name}")

        print("Downloading result file content...")
        file_content = client.files.download(file=result_file_name)
        # Process file_content (bytes) as needed
        
        return file_content

    
    else:
        print("No results found (neither file nor inline).")
        return None

In [ ]:
result = fetch_and_parse_batch_results(client, "batches/ixfnf86mx2e554pe5uuo2ow8viopkgh7qwlx")

Batch job is not ready. Current state: JOB_STATE_PENDING
